In [1]:
import os
import logging
from transformers import logging as transformers_logging
import tensorflow as tf
from mongo_db_client import MongoDbClient
from embedding_dataset import EmbeddingDataset
import numpy as np
from embedding_generator import EmbeddingGenerator

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
transformers_logging.set_verbosity_error()

/Users/beto/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
with EmbeddingDataset(dataset_name='embeddings', dataset_dir='../datasets/embeddings/') as embedding_dataset:
  embeddings = list(embedding_dataset.list())

embedding = embeddings[0]
print(f"{embedding['id']} {embedding['code_embedding'].shape} {embedding['comment_embedding'].shape}")


64eaeec92e9e80eac1b046e8 (480,) (480,)
